In [30]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jwpho\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jwpho\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [32]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [33]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

325 documents
101 classes ['goodbye', 'greeting', 'negative', 'positive', 'problem:3M Maintenance', 'problem:4 axis clamp', 'problem:6M Maintenance', 'problem:ATC LS error', 'problem:BUE', 'problem:Burr formation', 'problem:Chipping', 'problem:Chipping1', 'problem:Chipping2', 'problem:Crater wear', 'problem:Door not close', 'problem:EDM', 'problem:Every week Maintenance', 'problem:Everyday Maintenance', 'problem:Flank wear 1', 'problem:Flank wear 2', 'problem:Flank wear 3', 'problem:Flank wear1', 'problem:Flank wear2', 'problem:Flank wear3', 'problem:Machine power', "problem:Machine's electric function", 'problem:PLC alarm', 'problem:Plastic deformation1', 'problem:Rotation direction', 'problem:T CODE = SP', 'problem:VMC1060', 'problem:VMC1270', 'problem:VMC1370', 'problem:VMC1580', 'problem:VMC1690', 'problem:VMC1890', 'problem:VMC540', 'problem:VMC650', 'problem:VMC850', 'problem:VMC960', 'problem:Vibration', 'problem:Vibration1', 'problem:Vibration2', 'problem:Vibration3', 'problem:

In [34]:

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [35]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
325/325 [==============================] - 0s 886us/step - loss: 4.5636 - accuracy: 0.0708
Epoch 2/200
325/325 [==============================] - 0s 460us/step - loss: 4.3772 - accuracy: 0.1138
Epoch 3/200
325/325 [==============================] - 0s 463us/step - loss: 4.2685 - accuracy: 0.1138
Epoch 4/200
325/325 [==============================] - 0s 472us/step - loss: 4.1594 - accuracy: 0.1138
Epoch 5/200
325/325 [==============================] - 0s 477us/step - loss: 4.0199 - accuracy: 0.1138
Epoch 6/200
325/325 [==============================] - 0s 476us/step - loss: 3.8748 - accuracy: 0.1262
Epoch 7/200
325/325 [==============================] - 0s 854us/step - loss: 3.7630 - accuracy: 0.1446
Epoch 8/200
325/325 [==============================] - 0s 910us/step - loss: 3.6100 - accuracy: 0.1446
Epoch 9/200
325/325 [==============================] - 0s 908us/step - loss: 3.5386 - accuracy: 0.1785
Epoch 10/200
325/325 [==============================] - 0s 877us/step - l

Epoch 158/200
325/325 [==============================] - 0s 662us/step - loss: 0.8161 - accuracy: 0.6646
Epoch 159/200
325/325 [==============================] - 0s 789us/step - loss: 0.8358 - accuracy: 0.6923
Epoch 160/200
325/325 [==============================] - 0s 965us/step - loss: 0.8314 - accuracy: 0.7046
Epoch 161/200
325/325 [==============================] - 0s 946us/step - loss: 0.6689 - accuracy: 0.7600
Epoch 162/200
325/325 [==============================] - 0s 880us/step - loss: 0.8394 - accuracy: 0.6954
Epoch 163/200
325/325 [==============================] - 0s 930us/step - loss: 0.8556 - accuracy: 0.6769
Epoch 164/200
325/325 [==============================] - 0s 949us/step - loss: 0.9308 - accuracy: 0.6708
Epoch 165/200
325/325 [==============================] - 0s 837us/step - loss: 0.9043 - accuracy: 0.6892
Epoch 166/200
325/325 [==============================] - 0s 774us/step - loss: 0.8428 - accuracy: 0.6892
Epoch 167/200
325/325 [==============================] 

In [36]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [37]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# If it's related to a particular issue, then it returns a response corresponding 
# to the most likely solution based on the 'confidence score'; if it's a general question
# then choose from the response list randomly.


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    if tag.startswith('problem:'):
        result = sorted(prob_sol_dict[tag], key = prob_sol_dict[tag].get, reverse = True)[0]
        print(result)
        return result
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

#Matching the negative and positive feedback patterns and return "positive" or 'negative' as a string

def getFeedback(feedback, intents_json): # Returns Pos or Neg
    ints = predict_class(feedback, model)
    res = getResponse(ints, intents)
    print(ints,res)
    return res
        

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res,ints


In [38]:
#updates the priority of each possible solution by incrementing or substracting 1 from the total count

def updatePriority(solution,prob_tag, label):
    if prob_tag == '':
        return
    if label == 'positive':
        prob_sol_dict[prob_tag][solution] += 1
        print(prob_sol_dict[prob_tag])
    if label == 'negative':
        prob_sol_dict[prob_tag][solution] -= 1
        print(prob_sol_dict[prob_tag])

In [39]:
# A new data structure to store the multiple solutions with their frequency count(confidence score)
# {problem_tag1: {solution1: count1, solution2: count2,...},
#  problem _tag2: ...}

prob_sol_dict = {}

for item in intents['intents']:
    if item['tag'].startswith('problem:'):
        count_dict = {}
        for solution in item['responses']:
            count_dict[solution] = 1
        prob_sol_dict[item['tag']] = count_dict
prob_sol_dict[''] = 'puck'
print(prob_sol_dict)

{'problem:umbrella tool': {'Set Feed Rate knob as 0 under HANDWHEEL model, press RESET to get the tool magazine back.': 1}, 'problem:arm tool': {'Set Feed Rate knob as 0 under HANDWHEEL model, press RESET to get the tool magazine back': 1}, 'problem:axis trouble': {'Clean the axis cover': 1, 'Change the bearing': 1, 'If it is from Z axis, check it is loosen for the hammer and guide way is loosen or not. Check if the hammer chain and guide wheel is wearing or not.': 1}, 'problem:spindle trouble': {'Lubricate the spindle belt': 1, 'Adjust the belt tension': 1, 'Replace the spindle belt or spindle.': 1}, 'problem:coolant pump': {'Check if the coolant liquid is enough or not.': 1, 'Input water to exclude the air inside the coolan tank': 1, 'Check the power sequence.': 1, 'Check the thermal relay': 1, 'Replace the coolant': 1}, 'problem:lubrication pump': {'Check if the lubrication oil is enough or not.': 1, 'Check the oil filter': 1, 'Check if the oil is SHELL TONA68 or equal one': 1, 'Adj

In [ ]:
#Creating GUI with tkinter

import smtplib
from smtplib import SMTPException
import tkinter
from tkinter import *
import tkinter.ttk as ttk
import webbrowser

current_solution = ''
current_prob_tag = ''
wrong_answer = 0
warranty_asked = False 

global user_email
global receiver_email
global subj
global password
global body

def send():
    global current_solution
    global current_prob_tag
    global wrong_answer
    global temp_sol_dict
    global warranty_asked
    
    end_of_problem = False
    passed = False   
        
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)


    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#ffffff", font=("Arial", 12 ))

        res, probs_dict = chatbot_response(msg)
        tag = probs_dict[0]['intent']
        
        
        if wrong_answer >0:
            
            if tag == 'positive':
                
                end_of_problem = True
                wrong_answer = 0
                
                if warranty_asked:
                    warranty_asked = False
                    ChatLog.insert(END, "Kwahz Bot {◕ᗜ◕}: Please consult your warranty and send your machine back if necessary" '\n\n')
                    
                else:
                    print('---Feedback running---')
                    updatePriority(current_solution, current_prob_tag, tag)

            if tag == 'negative':
                
                if warranty_asked:
                    ChatLog.insert(END, "Kwahz Bot {◕0◕}: Please give us a feedback via email" '\n\n')
                    end_of_problem = True
                    wrong_answer = 0
                    passed = True
                    warranty_asked = False
                else:    
                    print('---Feedback running---')
                    updatePriority(current_solution, current_prob_tag, tag)
                    wrong_answer += 1
                    temp_sol_dict.pop(current_solution)
                    passed = True
                    
                    if not temp_sol_dict:
                        ChatLog.insert(END, "Kwahz Bot ¯\(°_◕)/¯: Do you have a warranty?" '\n\n')
                        warranty_asked = True
                    else:
                        current_solution = sorted(temp_sol_dict,key = temp_sol_dict.get, reverse = True)[0]
                        print(current_solution)
                        ChatLog.insert(END, "Kwahz Bot {◕.◕}: " + current_solution + '\n\n')
                

        #Identifies problems from a message
        elif tag.startswith('problem:'):
            print(tag)
            current_solution = res
            current_prob_tag = tag
            print(current_solution)

        # Identifies feedback from a message
        elif tag == 'positive':
            print('---Feedback running---')
            updatePriority(current_solution, current_prob_tag, tag)
            end_of_problem = True
            wrong_answer = 0

        elif tag == 'negative':
            if current_solution != '':
                print('---Feedback running---')
                updatePriority(current_solution, current_prob_tag, tag)
                wrong_answer += 1

                # Now assign the alternative solution
                temp_sol_dict = prob_sol_dict[current_prob_tag].copy()
                temp_sol_dict.pop(current_solution)

                #Checking empty dictionary
                if not temp_sol_dict:
                    ChatLog.insert(END, "Kwahz Bot {◕-◕}: Please give us a feedback via email" '\n\n')
                    end_of_problem = True
                else:
                    current_solution = sorted(temp_sol_dict,key = temp_sol_dict.get, reverse = True)[0]
                    print(current_solution)
                    ChatLog.insert(END, "Kwahz Bot {◕3◕}: " + current_solution + '\n\n')
                    passed = True
                
        if not passed:
            ChatLog.insert(END, "Kwahz Bot {◕ᗜ◕}: " + res + '\n\n')
        if end_of_problem:
            current_solution = ''
            current_prob_tag = ''
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
ttk.Style().theme_use('alt')
base.title("Chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
base.iconbitmap("C:\\Users\\jwpho\\bot.ico")

def gmail():   
    usermail = user_email.get()
    receivermail=receiver_email.get()           
    server=smtplib.SMTP('smtp.gmail.com:587')
    pass_word=password.get()
    subject=subj.get()
    #This allow you to include a subject by adding from, to and subject 
    line
    main_message=body.get('1.0', 'end-1c')
    Body="""From: Name here <usermail>
    To: <receivermail>
    Subject:%s 

    %s
    """ %(subject,main_message )

    try:
            server=smtplib.SMTP('smtp.gmail.com:587')
            server.ehlo()
            server.starttls()
            server.login(usermail, pass_word  )
            server.sendmail(usermail,receivermail, Body )

            text.insert(1.0, 'message sent')
         #error handling
    except  (smtplib.SMTPException,ConnectionRefusedError,OSError):
            text.insert(1.0, 'message not sent')

            
def openNewWindow(): 
      
    # Toplevel object which will  
    # be treated as a new window 
       
    root= Tk(className="Gmail")
    root.config(bg="#ffffff")
    root.geometry("450x500")
    root.resizable(width=FALSE, height=FALSE)
    root.iconbitmap("C:\\Users\\jwpho\\gmail.ico")
     
    
    #user mail
    user_email = Label(root, font=("Open Sans", 13, 'bold'), relief=FLAT, text="Sender Gmail:")
    user_email.pack()
    user_email.config(bg="#ffffff", fg="#000000")
    user_email = Entry(root, relief=FLAT, bd =8, bg="#e6e6e6")
    user_email.pack(fill=X)
    
    #receiver email
    receiver_email = Label(root, font = ("Open Sans", 13, 'bold'), relief=FLAT, text="Recipient email:")
    receiver_email.pack()
    receiver_email.config(bg="#ffffff", fg="#000000")
    receiver_email = Entry(root, relief=FLAT, bd =8, bg="#e6e6e6")
    receiver_email.pack(fill=X)
    
    #subject line
    subj= Label(root, font = ("Open Sans", 13, 'bold'), relief=FLAT, text="Subject:")
    subj.pack()
    subj.config(bg="#ffffff", fg="#000000")
    
    subj = Entry(root, relief=FLAT, bd =8, bg="#e6e6e6")
    subj.pack(fill=X)

    #Body of the message
    body = Text(root, font=("Open Sans", 13, 'bold'),  relief=FLAT , bd=8)
    body.config(bg="#525252", fg='#ffffff', height=10)
    body.pack(fill=BOTH, expand=True)
    
    #password widget
    password = Label(root,
                     font=("Open Sans", 13, 'bold'), relief=FLAT, text="Password:")
    password.pack()
    password.config(bg="#ffffff", fg="#000000")
    
    password= Entry(root, show='*', relief=FLAT, bd =8, bg="#e6e6e6")
    password.pack(fill=X)

    #submit button
    submit_mail = Button(root, font=("Arial Narrow",13,'bold'), relief = FLAT, text="Send", bd=8, bg="#ffffff", activebackground="#919191", fg='#000000', command=gmail)
    submit_mail.pack(fill=X)
    
            
new = 1
url = "https://www.google.com"

def openweb():
    webbrowser.open(url,new=new)

#Create Chat window
ChatLog = Text(base, bd=0, bg="#525252", fg='#ffffff', height="8", width="50", font="Calibri")

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview)
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Arial Narrow",12,'bold'), text="Send", width="14", height=5,
                    bd=0, bg="#ffffff", activebackground="#919191",fg='#000000',
                    command= send)

EmailButton = Button(base, font=("Arial Narrow",12,'bold'), text="Email", width="12", height=5,
                    bd=0, bg="#ffffff", activebackground="#919191",fg='#000000',
                    command= openNewWindow)
WebsiteButton = Button(base, font=("Arial Narrow",12,'bold'), text="Visit our website", width="12", height=5,
                    bd=0, bg="#ffffff", activebackground="#919191",fg='#000000',
                    command=openweb)

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="#525252",fg='#ffffff',width="29", height="5", font=("Arial", 10))
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=36, height=416, width=10)
ChatLog.place(x=6,y=36, height=416, width=370)
EntryBox.place(x=6, y=456, height=35, width=248)
SendButton.place(x=260, y=456, height=35)
EmailButton.place(x=320, y=10, height=20, width=50)
WebsiteButton.place(x=6, y=10, height=20, width=150)

base.mainloop()

---Feedback running---
---Feedback running---
Z axis can't reach ATC - check if the 2nd home point of Z axis signal F96.2 is right or not
problem:PLC alarm
Z axis can't reach ATC - check if the 2nd home point of Z axis signal F96.2 is right or not
---Feedback running---
{"Z axis can't reach ATC - check if the 2nd home point of Z axis signal F96.2 is right or not": 2}


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\jwpho\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-40-c80042ef1cbe>", line 40, in send
    res, probs_dict = chatbot_response(msg)
  File "<ipython-input-37-da8efe08e128>", line 64, in chatbot_response
    res = getResponse(ints, intents)
  File "<ipython-input-37-da8efe08e128>", line 41, in getResponse
    tag = ints[0]['intent']
IndexError: list index out of range
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\jwpho\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-40-c80042ef1cbe>", line 40, in send
    res, probs_dict = chatbot_response(msg)
  File "<ipython-input-37-da8efe08e128>", line 64, in chatbot_response
    res = getResponse(ints, intents)
  File "<ipython-input-37-da8efe08e128>", line 41, in getResponse
    tag = ints[0]['intent']
Inde